In [ ]:
import random
# Defining Function for writing in files (we will use to convert lists to .txt files)
def write_files(file_path, lst):
    with open(file_path, 'w', encoding='utf-8') as file:
        for item in lst:
            file.write("%s\n" % item)

# Defining Function which will append eachsentence as a string in the list of string 
def read_files(file_paths):
  
    hindi_sentences = []
    english_sentences = []
    gujarati_sentences = []

    # Reading each file and extracting the sentences
    for file_path in file_paths:
        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()
            if file_path.lower().endswith('.hi'):
                hindi_sentences.extend([line.strip() for line in lines])
            elif file_path.lower().endswith('.en'):
                english_sentences.extend([line.strip() for line in lines])
            elif file_path.lower().endswith('.gu'):
                gujarati_sentences.extend([line.strip() for line in lines])
    return hindi_sentences, english_sentences, gujarati_sentences

def select_random_sentences(hindi_sentences, english_sentences, gujarati_sentences, size):
   
    random_hindi_sentences_1k = []
    random_english_sentences_1k = []
    random_gujarati_sentences_1k = []

    for _ in range(size):
        random_index = random.randint(0, len(hindi_sentences) - 1)
        random_hindi_sentences_1k.append(hindi_sentences[random_index])
        random_english_sentences_1k.append(english_sentences[random_index])
        random_gujarati_sentences_1k.append(gujarati_sentences[random_index])
    
    return random_hindi_sentences_1k, random_english_sentences_1k, random_gujarati_sentences_1k

# Example usage:
file_paths = ("/kaggle/input/hindi-test/test.hi", "/kaggle/input/hindi-test/test.en", "/kaggle/input/hindi-test/test.gu")
hindi_sentences, english_sentences, gujarati_sentences = read_files(file_paths)

# Number of strings in the list
size = 1000
hi_sen_1k, en_sen_1k, gu_sen_1k = select_random_sentences(hindi_sentences, english_sentences, gujarati_sentences, size)

# Printing the sentences in each language one by one
print("Hindi Sentences :-", hi_sen_1k)
print("English Sentences :-", en_sen_1k)
print("Gujarati Sentences :-", gu_sen_1k)


In [2]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def translate(src_lang, tgt_lng, l):
    tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M", src_lang=src_lang)
    # Move the model to the device
    model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M").to(device)

    temp = []
    for i in range(len(l)):
        inputs = tokenizer(l[i], return_tensors="pt").to(device)

        # Move inputs to the same device as the model
        inputs = {key: val.to(device) for key, val in inputs.items()}

        translated_tokens = model.generate(
            **inputs, forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lng], max_length=30
        ) 
        # Move generated tokens back to CPU before decoding
        translated_tokens = translated_tokens.to('cpu')
        temp.append(tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0])
    return temp


In [3]:
# Translate Hindi to English
hi_to_en_translations = translate('hin_Deva', 'eng_Latn', hi_sen_1k)
write_files("hindi_to_english.txt", hi_to_en_translations)

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

the `lang_code_to_id` attribute is deprecated. The logic is natively handled in the `tokenizer.adder_tokens_decoder` this attribute will be removed in `transformers` v4.38


In [4]:
# Translate English to Hindi
en_to_hi_translations = translate('eng_Latn', 'hin_Deva', en_sen_1k)
write_files("english_to_hindi.txt", en_to_hi_translations)

In [5]:
# Translate Hindi to Gujarati
hi_to_gu_translations = translate('hin_Deva', 'guj_Gujr', hi_sen_1k)
write_files("hindi_to_gujarati.txt", hi_to_gu_translations)

In [6]:
# Translate Gujarati to Hindi
gu_to_hi_translations = translate('guj_Gujr', 'hin_Deva', gu_sen_1k)
write_files("gujarati_to_hindi.txt", gu_to_hi_translations)

In [7]:
write_files("ground_truth_hin.txt", hi_sen_1k)
write_files("ground_truth_eng.txt", en_sen_1k)
write_files("ground_truth_guj.txt", gu_sen_1k)

In [3]:
hi_sen_50 = hi_sen_1k[:50]
en_sen_50 = en_sen_1k[:50]
gu_sen_50 = gu_sen_1k[:50]

In [5]:
print(hi_sen_50)

['उन्\u200dहोंने कहा कि इस रचनात्\u200dमक माहौल का इस्\u200dतेमाल तेजी से नए भारत के निर्माण में किया जाना चाहिए।', 'इसे हमारी लोकतांत्रिक परंपराओं से पंख मिले हैं, जो धन एवं मूल्य सृजन दोनों पर जोर देते हुए उद्यमशीलता की भावना को मजबूत करती है और अपनी अर्थव्यवस्था की समृद्धि के लिए आधुनिकीकरण की तलाश करती है।', 'हर वर्ष इन दोनों ही स्थानों पर बड़ी संख्या में तीर्थयात्री जाते हैं।', 'प्रधानमंत्री ने पेरिस जलवायु समझौते को पूरी दुनिया की साझा विरासत बताया किया और उम्मीद जताई कि यह अगली पीढ़ी के लिए उम्मीद की नई किरण बनेगा।', 'उन्\u200dहोंने देश की सेवा करते हुए अपने प्राण न्\u200dयोच्\u200dछावर किए हैं।', 'विश्व बैंक और आईएमएफ के आकलन के अनुसार, यह तेजी जारी रहेगी।', 'उन्होंने कहा कि केंद्र सरकार ने पिछले 3 वर्षों में पांच करोड़ शौचालयों का निर्माण किया है।', 'उन्होंने कहा कि प्रधानमंत्री कृषि सिंचाई योजना में प्रत्येक खेत के लिए जल की परिकल्पना की गई है।', 'प्रधानमंत्री ने रेलवे स्टेशनों के पुनर्विकास से संबंधित कार्यों को तेज करने और गैर किराया राजस्व के क्षेत्र में रचनात्मक काम करने 

In [4]:
write_files("hi_sen_50.txt", hi_sen_50)
write_files("en_sen_50.txt", en_sen_50)
write_files("gu_sen_50.txt", gu_sen_50)